# Airfoil's Aerodynamic Coefficient Prediction using ANNs

In [1]:
import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'

In [2]:
# importing the dependencies
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from tensorflow import keras
from keras import Input
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras.metrics import RootMeanSquaredError
from keras.callbacks import ReduceLROnPlateau
import matplotlib.pyplot as plt

In [3]:
# reading the csv file
df = pd.read_csv("/home/nevilcp/ML_Aero/results/NACA4D_05/NACA4D_05.csv")

In [4]:
# printing the first 10 rows of the dataset
df.head(10)

,t,m,p,yU1,yU2,yU3,yU4,yU5,yL1,yL2,yL3,yL4,yL5,alpha,M,Re,CL,CD,Cm
0,5,0,5,0.016694,0.024736,0.022058,0.013163,0.004284,-0.016694,-0.024736,-0.022058,-0.013163,-0.004284,-10.0,0.2,100000,-1.158,0.02657,0.005
1,5,0,5,0.016694,0.024736,0.022058,0.013163,0.004284,-0.016694,-0.024736,-0.022058,-0.013163,-0.004284,-9.0,0.2,100000,-1.043,0.02194,0.004
2,5,0,5,0.016694,0.024736,0.022058,0.013163,0.004284,-0.016694,-0.024736,-0.022058,-0.013163,-0.004284,-8.0,0.2,100000,-0.928,0.01876,0.004
3,5,0,5,0.016694,0.024736,0.022058,0.013163,0.004284,-0.016694,-0.024736,-0.022058,-0.013163,-0.004284,-7.0,0.2,100000,-0.813,0.01463,0.004
4,5,0,5,0.016694,0.024736,0.022058,0.013163,0.004284,-0.016694,-0.024736,-0.022058,-0.013163,-0.004284,-6.0,0.2,100000,-0.697,0.02806,0.003
5,5,0,5,0.016694,0.024736,0.022058,0.013163,0.004284,-0.016694,-0.024736,-0.022058,-0.013163,-0.004284,-5.0,0.2,100000,-0.581,0.01660,0.003
6,5,0,5,0.016694,0.024736,0.022058,0.013163,0.004284,-0.016694,-0.024736,-0.022058,-0.013163,-0.004284,-4.0,0.2,100000,-0.465,0.01146,0.002
7,5,0,5,0.016694,0.024736,0.022058,0.013163,0.004284,-0.016694,-0.024736,-0.022058,-0.013163,-0.004284,-3.0,0.2,100000,-0.349,0.00893,0.002
8,5,0,5,0.016694,0.024736,0.022058,0.013163,0.004284,-0.016694,-0.024736,-0.022058,-0.013163,-0.004284,-2.0,0.2,100000,-0.233,0.01393,0.001
9,5,0,5,0.016694,0.024736,0.022058,0.013163,0.004284,-0.016694,-0.024736,-0.022058,-0.013163,-0.004284,-1.0,0.2,100000,-0.116,0.01358,0.001


In [5]:
# checking for missing values in the dataset
df.isnull().sum()

t        0
m        0
p        0
yU1      0
yU2      0
yU3      0
yU4      0
yU5      0
yL1      0
yL2      0
yL3      0
yL4      0
yL5      0
alpha    0
M        0
Re       0
CL       0
CD       0
Cm       0
dtype: int64

In [6]:
# description of the dataset
df.describe()

,t,m,p,yU1,yU2,yU3,yU4,yU5,yL1,yL2,yL3,yL4,yL5,alpha,M,Re,CL,CD,Cm
count,176400.000000,176400.000000,176400.000000,176400.000000,176400.000000,176400.000000,176400.000000,176400.000000,176400.000000,176400.000000,176400.000000,176400.000000,176400.000000,176400.000000,176400.00000,176400.000000,176400.000000,176400.000000,176400.000000
mean,20.000000,4.500000,40.000000,0.090151,0.136055,0.129532,0.085117,0.029149,-0.041429,-0.062016,-0.047469,-0.020889,-0.005748,0.000000,0.20000,300000.000000,0.692283,0.026646,-0.134151
std,10.000028,2.872289,22.912943,0.041158,0.055584,0.051843,0.035541,0.013507,0.036319,0.055251,0.051423,0.034288,0.012233,6.055318,0.08165,141421.757093,0.926925,0.010960,0.113007
min,5.000000,0.000000,5.000000,0.016694,0.024736,0.022058,0.013163,0.004284,-0.116860,-0.173149,-0.154409,-0.092144,-0.029989,-10.000000,0.10000,100000.000000,-1.489000,0.001800,-0.514000
25%,10.000000,2.000000,22.500000,0.056055,0.093189,0.089629,0.058491,0.019082,-0.067850,-0.104319,-0.087356,-0.046298,-0.014602,-5.000000,0.10000,200000.000000,-0.011000,0.018990,-0.199000
50%,20.000000,4.500000,40.000000,0.089551,0.135782,0.129491,0.084496,0.028075,-0.040627,-0.062004,-0.047239,-0.020962,-0.006221,-0.000000,0.20000,300000.000000,0.691000,0.025390,-0.106000
75%,30.000000,7.000000,57.500000,0.121686,0.179612,0.167965,0.109744,0.037234,-0.014891,-0.018975,-0.008716,0.003164,0.002137,5.000000,0.30000,400000.000000,1.379000,0.032540,-0.046000
max,35.000000,9.000000,75.000000,0.199249,0.262026,0.245063,0.182098,0.080134,0.073382,0.065108,0.067199,0.076689,0.036850,10.000000,0.30000,500000.000000,3.599000,0.235470,0.025000


In [7]:
# shuffling all the rows
df = df.sample(frac=1)
df.head(10)

,t,m,p,yU1,yU2,yU3,yU4,yU5,yL1,yL2,yL3,yL4,yL5,alpha,M,Re,CL,CD,Cm
126470,15,7,15,0.106154,0.143124,0.125120,0.075748,0.023948,-0.001078,-0.005048,-0.007866,-0.004074,-0.002099,-2.0,0.1,300000,0.558,0.01633,-0.109
13454,5,0,65,0.016694,0.024736,0.022058,0.013163,0.004284,-0.016694,-0.024736,-0.022058,-0.013163,-0.004284,4.0,0.3,100000,0.478,0.01259,-0.002
54126,20,3,5,0.093112,0.127308,0.112430,0.067379,0.021615,-0.028660,-0.069902,-0.065087,-0.038884,-0.012957,-1.0,0.3,300000,0.235,0.01424,-0.031
122322,20,6,75,0.081477,0.133259,0.141155,0.112523,0.046791,-0.053175,-0.065707,-0.035485,0.007435,0.010764,8.0,0.2,500000,2.336,0.03775,-0.339
113854,25,6,35,0.115624,0.179596,0.167971,0.105304,0.034322,-0.052800,-0.068585,-0.053078,-0.027426,-0.009049,3.0,0.2,200000,1.287,0.03192,-0.159
72653,35,4,5,0.140817,0.209406,0.189718,0.114987,0.037221,-0.013847,-0.132913,-0.124432,-0.074053,-0.024275,4.0,0.3,500000,1.173,0.02941,-0.044
97904,15,5,45,0.067362,0.114870,0.115855,0.075278,0.024705,-0.034233,-0.034255,-0.016518,-0.004078,-0.001261,-8.0,0.3,300000,-0.306,0.02036,-0.149
122633,25,6,75,0.100289,0.158274,0.162959,0.125692,0.051930,-0.067604,-0.090311,-0.057860,-0.005624,0.006896,4.0,0.1,500000,1.869,0.03210,-0.333
20100,5,1,15,0.023768,0.034594,0.030369,0.018189,0.005803,-0.009703,-0.014877,-0.013749,-0.008140,-0.002767,-7.0,0.3,300000,-0.727,0.01534,-0.014
17664,5,1,5,0.026690,0.034290,0.029821,0.017742,0.005649,-0.006699,-0.015182,-0.014297,-0.008587,-0.002920,-7.0,0.1,200000,-0.698,0.01447,-0.009


In [8]:
# dropping columns t, m & p
df = df.drop(columns=['t', 'm', 'p'], axis=1)
df.head(10)

,yU1,yU2,yU3,yU4,yU5,yL1,yL2,yL3,yL4,yL5,alpha,M,Re,CL,CD,Cm
126470,0.106154,0.143124,0.125120,0.075748,0.023948,-0.001078,-0.005048,-0.007866,-0.004074,-0.002099,-2.0,0.1,300000,0.558,0.01633,-0.109
13454,0.016694,0.024736,0.022058,0.013163,0.004284,-0.016694,-0.024736,-0.022058,-0.013163,-0.004284,4.0,0.3,100000,0.478,0.01259,-0.002
54126,0.093112,0.127308,0.112430,0.067379,0.021615,-0.028660,-0.069902,-0.065087,-0.038884,-0.012957,-1.0,0.3,300000,0.235,0.01424,-0.031
122322,0.081477,0.133259,0.141155,0.112523,0.046791,-0.053175,-0.065707,-0.035485,0.007435,0.010764,8.0,0.2,500000,2.336,0.03775,-0.339
113854,0.115624,0.179596,0.167971,0.105304,0.034322,-0.052800,-0.068585,-0.053078,-0.027426,-0.009049,3.0,0.2,200000,1.287,0.03192,-0.159
72653,0.140817,0.209406,0.189718,0.114987,0.037221,-0.013847,-0.132913,-0.124432,-0.074053,-0.024275,4.0,0.3,500000,1.173,0.02941,-0.044
97904,0.067362,0.114870,0.115855,0.075278,0.024705,-0.034233,-0.034255,-0.016518,-0.004078,-0.001261,-8.0,0.3,300000,-0.306,0.02036,-0.149
122633,0.100289,0.158274,0.162959,0.125692,0.051930,-0.067604,-0.090311,-0.057860,-0.005624,0.006896,4.0,0.1,500000,1.869,0.03210,-0.333
20100,0.023768,0.034594,0.030369,0.018189,0.005803,-0.009703,-0.014877,-0.013749,-0.008140,-0.002767,-7.0,0.3,300000,-0.727,0.01534,-0.014
17664,0.026690,0.034290,0.029821,0.017742,0.005649,-0.006699,-0.015182,-0.014297,-0.008587,-0.002920,-7.0,0.1,200000,-0.698,0.01447,-0.009


In [9]:
# defining the feature and target columns
X = df.drop(columns=['CL', 'CD', 'Cm'])
y = df[['CL', 'CD', 'Cm']]

In [10]:
# displaying the features table
X.head(10)

,yU1,yU2,yU3,yU4,yU5,yL1,yL2,yL3,yL4,yL5,alpha,M,Re
126470,0.106154,0.143124,0.125120,0.075748,0.023948,-0.001078,-0.005048,-0.007866,-0.004074,-0.002099,-2.0,0.1,300000
13454,0.016694,0.024736,0.022058,0.013163,0.004284,-0.016694,-0.024736,-0.022058,-0.013163,-0.004284,4.0,0.3,100000
54126,0.093112,0.127308,0.112430,0.067379,0.021615,-0.028660,-0.069902,-0.065087,-0.038884,-0.012957,-1.0,0.3,300000
122322,0.081477,0.133259,0.141155,0.112523,0.046791,-0.053175,-0.065707,-0.035485,0.007435,0.010764,8.0,0.2,500000
113854,0.115624,0.179596,0.167971,0.105304,0.034322,-0.052800,-0.068585,-0.053078,-0.027426,-0.009049,3.0,0.2,200000
72653,0.140817,0.209406,0.189718,0.114987,0.037221,-0.013847,-0.132913,-0.124432,-0.074053,-0.024275,4.0,0.3,500000
97904,0.067362,0.114870,0.115855,0.075278,0.024705,-0.034233,-0.034255,-0.016518,-0.004078,-0.001261,-8.0,0.3,300000
122633,0.100289,0.158274,0.162959,0.125692,0.051930,-0.067604,-0.090311,-0.057860,-0.005624,0.006896,4.0,0.1,500000
20100,0.023768,0.034594,0.030369,0.018189,0.005803,-0.009703,-0.014877,-0.013749,-0.008140,-0.002767,-7.0,0.3,300000
17664,0.026690,0.034290,0.029821,0.017742,0.005649,-0.006699,-0.015182,-0.014297,-0.008587,-0.002920,-7.0,0.1,200000


In [11]:
# displaying the targets table
y.head(10)

,CL,CD,Cm
126470,0.558,0.01633,-0.109
13454,0.478,0.01259,-0.002
54126,0.235,0.01424,-0.031
122322,2.336,0.03775,-0.339
113854,1.287,0.03192,-0.159
72653,1.173,0.02941,-0.044
97904,-0.306,0.02036,-0.149
122633,1.869,0.03210,-0.333
20100,-0.727,0.01534,-0.014
17664,-0.698,0.01447,-0.009


## Splitting the Dataset

In [12]:
# splitting dataset into training, validation & testing dataset
X_train, X_temp, y_train, y_temp = train_test_split(X, y, train_size=0.7, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [13]:
# normalizing the training, validation & testing datasets
X_scaler = StandardScaler()
X_train = X_scaler.fit_transform(X_train)
X_val = X_scaler.transform(X_val)
X_test = X_scaler.transform(X_test)

In [14]:
# displaying X_train, X_val & X_test after standardization
display(pd.DataFrame(X_train).head())
display(pd.DataFrame(X_val).head())
display(pd.DataFrame(X_test).head())

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0.942691,0.844397,0.648360,0.355252,0.193427,-1.693382,-1.836047,-1.916978,-1.921675,-1.839159,0.993088,-1.227464,0.705920
1,1.318486,1.011721,0.796905,0.533899,0.313452,0.557773,0.143013,-0.086295,-0.261653,-0.372015,-0.329033,-0.002549,-0.001312
2,-0.998649,-0.828903,-0.689770,-0.656906,-0.661736,0.618266,0.950207,1.029417,0.866207,0.704307,0.001498,-0.002549,-0.001312
3,-0.183988,-0.405010,-0.627579,-0.747074,-0.823860,2.318049,2.282678,1.947090,1.538616,1.227588,-0.494298,-0.002549,1.413152
4,-1.012692,-1.202103,-1.325767,-1.369418,-1.296253,0.620228,0.583460,0.387762,0.134285,0.018201,-0.990093,-0.002549,1.413152


,0,1,2,3,4,5,6,7,8,9,10,11,12
0,-0.495530,-0.189723,0.317523,1.244979,2.042685,0.244848,0.673245,1.179270,2.080410,2.748794,0.827823,1.222366,-1.415776
1,-0.734475,-0.657173,-0.493575,-0.240743,-0.146096,0.011296,0.227707,0.368516,0.533662,0.560515,-1.320623,-0.002549,-0.708544
2,-0.661612,-1.041544,-1.194626,-1.262247,-1.214796,1.061499,0.746664,0.517017,0.241228,0.104187,-0.329033,1.222366,-1.415776
3,-1.332402,-1.456356,-1.476237,-1.373437,-1.176457,0.271631,0.327012,0.236475,0.131367,0.150154,-1.485888,-1.227464,-0.708544
4,-0.446928,-0.279529,-0.303471,-0.390599,-0.456097,0.301832,0.607562,0.558335,0.370393,0.229573,0.993088,-0.002549,-0.001312


,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0.851452,0.968585,1.020687,0.885715,0.703342,-0.943867,-0.835025,-0.683445,-0.624303,-0.616017,1.323619,-1.227464,-0.708544
1,1.487703,0.877613,0.602068,0.330070,0.109183,2.075715,0.903003,0.570527,0.289375,0.095683,1.654149,-0.002549,0.705920
2,-0.554556,-0.446297,-0.487404,-0.570341,-0.607602,0.198914,0.442576,0.373802,0.188441,0.069249,-0.329033,-1.227464,-0.001312
3,-0.722505,-0.673683,-0.646661,-0.680982,-0.682382,0.024855,0.214126,0.214675,0.077364,-0.009546,1.654149,-1.227464,-1.415776
4,0.831541,0.852613,0.684329,0.459914,0.266837,-0.062244,-0.026969,-0.177702,-0.310521,-0.397990,-1.485888,-0.002549,1.413152


In [15]:
# displaying y_train, y_val & y_test after standardization
display(pd.DataFrame(y_train).head())
display(pd.DataFrame(y_val).head())
display(pd.DataFrame(y_test).head())

,CL,CD,Cm
22024,1.004,0.03371,-0.026
127100,0.644,0.02090,-0.102
97492,0.678,0.01635,-0.153
161161,0.583,0.03032,-0.148
37993,-0.505,0.01526,-0.028


,CL,CD,Cm
175155,2.506,0.05018,-0.497
84548,-0.291,0.02046,-0.167
53453,0.080,0.01712,-0.034
33412,-0.869,0.02072,-0.046
95713,1.414,0.02535,-0.135


,CL,CD,Cm
98739,1.915,0.04392,-0.170
159893,2.371,0.04323,-0.084
77855,0.272,0.01729,-0.100
62390,1.654,0.03389,-0.103
111700,-0.401,0.02394,-0.108


In [16]:
# defining the learning rate reduction callback
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=5, min_lr=0.00001)

## Network Performance Due to Multiple Hidden Layers

### Case 1: Model [64, 3]

In [17]:
# RMSE and R² metric scores on test data for Model M_64
M_64_metrics = []

for i in range(20):
    print(f"\nRun {i+1}/20 for Model M_64...")

    M_64 = Sequential([
        Input(shape=(X_train.shape[1],)),
        Dense(64, activation='relu'),
        Dense(y_train.shape[1], activation='linear')
    ])

    M_64.compile(
        optimizer=Adam(learning_rate=5e-4, beta_1=0.9, beta_2=0.999),
        loss='mse',
        metrics=[RootMeanSquaredError()]
    )

    M_64.fit(
        X_train, y_train,
        epochs=50, batch_size=128,
        validation_data=(X_val, y_val),
        callbacks=[reduce_lr],
        verbose=0
    )

    y_pred = M_64.predict(X_test, verbose=0)
    r2 = r2_score(y_test, y_pred, multioutput='raw_values')
    rmse = np.sqrt(mean_squared_error(y_test, y_pred, multioutput='raw_values'))

    M_64_metrics.append({
        'CL_rmse': rmse[0], 'CD_rmse': rmse[1], 'Cm_rmse': rmse[2],
        'CL_r2': r2[0], 'CD_r2': r2[1], 'Cm_r2': r2[2]
    })

results_M_64_df = pd.DataFrame(M_64_metrics)

print(results_M_64_df.mean())



Run 1/20 for Model M_64...


I0000 00:00:1764100483.746045   19592 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1193 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9
I0000 00:00:1764100485.611928   21023 device_compiler.h:196] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.



Run 2/20 for Model M_64...

Run 3/20 for Model M_64...

Run 4/20 for Model M_64...

Run 5/20 for Model M_64...

Run 6/20 for Model M_64...

Run 7/20 for Model M_64...

Run 8/20 for Model M_64...

Run 9/20 for Model M_64...

Run 10/20 for Model M_64...

Run 11/20 for Model M_64...

Run 12/20 for Model M_64...

Run 13/20 for Model M_64...

Run 14/20 for Model M_64...

Run 15/20 for Model M_64...

Run 16/20 for Model M_64...

Run 17/20 for Model M_64...

Run 18/20 for Model M_64...

Run 19/20 for Model M_64...

Run 20/20 for Model M_64...
CL_rmse    0.004378
CD_rmse    0.004406
Cm_rmse    0.002037
CL_r2      0.999977
CD_r2      0.841861
Cm_r2      0.999674
dtype: float32


### Case 2: Model [64, 32, 3]

In [18]:
# RMSE and R² metric scores on test data for Model M_64_32
M_64_32_metrics = []

for i in range(20):
    print(f"\nRun {i+1}/20 for Model M_64_32...")
    
    M_64_32 = Sequential([
        Input(shape=(X_train.shape[1],)),
        Dense(64, activation='relu'),
        Dense(32, activation='relu'),
        Dense(y_train.shape[1], activation='linear')
    ])

    M_64_32.compile(
        optimizer=Adam(learning_rate=5e-4, beta_1=0.9, beta_2=0.999),
        loss='mse',
        metrics=[RootMeanSquaredError()]
    )
    
    M_64_32.fit(
        X_train, y_train,
        epochs=50, batch_size=128,
        validation_data=(X_val, y_val),
        callbacks=[reduce_lr],
        verbose=0
    )

    y_pred = M_64_32.predict(X_test, verbose=0)
    r2 = r2_score(y_test, y_pred, multioutput='raw_values')
    rmse = np.sqrt(mean_squared_error(y_test, y_pred, multioutput='raw_values'))
    
    M_64_32_metrics.append({
        'CL_rmse': rmse[0], 'CD_rmse': rmse[1], 'Cm_rmse': rmse[2],
        'CL_r2': r2[0], 'CD_r2': r2[1], 'Cm_r2': r2[2]
    })

results_M_64_32_df = pd.DataFrame(M_64_32_metrics)

print(results_M_64_32_df.mean())



Run 1/20 for Model M_64_32...

Run 2/20 for Model M_64_32...

Run 3/20 for Model M_64_32...

Run 4/20 for Model M_64_32...

Run 5/20 for Model M_64_32...

Run 6/20 for Model M_64_32...

Run 7/20 for Model M_64_32...

Run 8/20 for Model M_64_32...

Run 9/20 for Model M_64_32...

Run 10/20 for Model M_64_32...

Run 11/20 for Model M_64_32...

Run 12/20 for Model M_64_32...

Run 13/20 for Model M_64_32...

Run 14/20 for Model M_64_32...

Run 15/20 for Model M_64_32...

Run 16/20 for Model M_64_32...

Run 17/20 for Model M_64_32...

Run 18/20 for Model M_64_32...

Run 19/20 for Model M_64_32...

Run 20/20 for Model M_64_32...
CL_rmse    0.003103
CD_rmse    0.004121
Cm_rmse    0.001734
CL_r2      0.999989
CD_r2      0.861625
Cm_r2      0.999763
dtype: float32


### Case 3: Model [64, 32, 16, 3]

In [19]:
# RMSE and R² metric scores on test data for Model M_64_32_16
M_64_32_16_metrics = []

for i in range(20):
    print(f"\nRun {i+1}/20 for Model M_64_32_16...")
    
    M_64_32_16 = Sequential([
        Input(shape=(X_train.shape[1],)),
        Dense(64, activation='relu'),
        Dense(32, activation='relu'),
        Dense(16, activation='relu'),
        Dense(y_train.shape[1], activation='linear')
    ])

    M_64_32_16.compile(
        optimizer=Adam(learning_rate=5e-4, beta_1=0.9, beta_2=0.999),
        loss='mse',
        metrics=[RootMeanSquaredError()]
    )
    
    M_64_32_16.fit(
        X_train, y_train,
        epochs=50, batch_size=128,
        validation_data=(X_val, y_val),
        callbacks=[reduce_lr],
        verbose=0
    )

    y_pred = M_64_32_16.predict(X_test, verbose=0)
    r2 = r2_score(y_test, y_pred, multioutput='raw_values')
    rmse = np.sqrt(mean_squared_error(y_test, y_pred, multioutput='raw_values'))
    
    M_64_32_16_metrics.append({
        'CL_rmse': rmse[0], 'CD_rmse': rmse[1], 'Cm_rmse': rmse[2],
        'CL_r2': r2[0], 'CD_r2': r2[1], 'Cm_r2': r2[2]
    })

results_M_64_32_16_df = pd.DataFrame(M_64_32_16_metrics)

print(results_M_64_32_16_df.mean())



Run 1/20 for Model M_64_32_16...

Run 2/20 for Model M_64_32_16...

Run 3/20 for Model M_64_32_16...

Run 4/20 for Model M_64_32_16...

Run 5/20 for Model M_64_32_16...

Run 6/20 for Model M_64_32_16...

Run 7/20 for Model M_64_32_16...

Run 8/20 for Model M_64_32_16...

Run 9/20 for Model M_64_32_16...

Run 10/20 for Model M_64_32_16...

Run 11/20 for Model M_64_32_16...

Run 12/20 for Model M_64_32_16...

Run 13/20 for Model M_64_32_16...

Run 14/20 for Model M_64_32_16...

Run 15/20 for Model M_64_32_16...

Run 16/20 for Model M_64_32_16...

Run 17/20 for Model M_64_32_16...

Run 18/20 for Model M_64_32_16...

Run 19/20 for Model M_64_32_16...

Run 20/20 for Model M_64_32_16...
CL_rmse    0.003109
CD_rmse    0.004088
Cm_rmse    0.001695
CL_r2      0.999989
CD_r2      0.863868
Cm_r2      0.999774
dtype: float32


### Case 4: Model [64, 32, 16, 8, 3]

In [20]:
# RMSE and R² metric scores on test data for Model M_64_32_16_8
M_64_32_16_8_metrics = []

for i in range(20):
    print(f"\nRun {i+1}/20 for Model M_64_32_16_8...")
    
    M_64_32_16_8 = Sequential([
        Input(shape=(X_train.shape[1],)),
        Dense(64, activation='relu'),
        Dense(32, activation='relu'),
        Dense(16, activation='relu'),
        Dense(8, activation='relu'),
        Dense(y_train.shape[1], activation='linear')
    ])

    M_64_32_16_8.compile(
        optimizer=Adam(learning_rate=5e-4, beta_1=0.9, beta_2=0.999),
        loss='mse',
        metrics=[RootMeanSquaredError()]
    )
    
    M_64_32_16_8.fit(
        X_train, y_train,
        epochs=50, batch_size=128,
        validation_data=(X_val, y_val),
        callbacks=[reduce_lr],
        verbose=0
    )

    y_pred = M_64_32_16_8.predict(X_test, verbose=0)
    r2 = r2_score(y_test, y_pred, multioutput='raw_values')
    rmse = np.sqrt(mean_squared_error(y_test, y_pred, multioutput='raw_values'))
    
    M_64_32_16_8_metrics.append({
        'CL_rmse': rmse[0], 'CD_rmse': rmse[1], 'Cm_rmse': rmse[2],
        'CL_r2': r2[0], 'CD_r2': r2[1], 'Cm_r2': r2[2]
    })

results_M_64_32_16_8_df = pd.DataFrame(M_64_32_16_8_metrics)

print(results_M_64_32_16_8_df.mean())



Run 1/20 for Model M_64_32_16_8...

Run 2/20 for Model M_64_32_16_8...

Run 3/20 for Model M_64_32_16_8...

Run 4/20 for Model M_64_32_16_8...

Run 5/20 for Model M_64_32_16_8...

Run 6/20 for Model M_64_32_16_8...

Run 7/20 for Model M_64_32_16_8...

Run 8/20 for Model M_64_32_16_8...

Run 9/20 for Model M_64_32_16_8...

Run 10/20 for Model M_64_32_16_8...

Run 11/20 for Model M_64_32_16_8...

Run 12/20 for Model M_64_32_16_8...

Run 13/20 for Model M_64_32_16_8...

Run 14/20 for Model M_64_32_16_8...

Run 15/20 for Model M_64_32_16_8...

Run 16/20 for Model M_64_32_16_8...

Run 17/20 for Model M_64_32_16_8...

Run 18/20 for Model M_64_32_16_8...

Run 19/20 for Model M_64_32_16_8...

Run 20/20 for Model M_64_32_16_8...
CL_rmse    0.003280
CD_rmse    0.004120
Cm_rmse    0.001757
CL_r2      0.999987
CD_r2      0.861719
Cm_r2      0.999756
dtype: float32


In [21]:
M_64_stats = results_M_64_df.mean()
M_64_32_stats = results_M_64_32_df.mean()
M_64_32_16_stats = results_M_64_32_16_df.mean()
M_64_32_16_8_stats = results_M_64_32_16_8_df.mean()

architectures = [
    '64, 3',
    '64, 32, 3',
    '64, 32, 16, 3',
    '64, 32, 16, 8, 3'
]

table_data = [
    (M_64_stats['CL_rmse'], M_64_stats['CD_rmse'], M_64_stats['Cm_rmse'],
     M_64_stats['CL_r2'], M_64_stats['CD_r2'], M_64_stats['Cm_r2']),
    
    (M_64_32_stats['CL_rmse'], M_64_32_stats['CD_rmse'], M_64_32_stats['Cm_rmse'],
     M_64_32_stats['CL_r2'], M_64_32_stats['CD_r2'], M_64_32_stats['Cm_r2']),
    
    (M_64_32_16_stats['CL_rmse'], M_64_32_16_stats['CD_rmse'], M_64_32_16_stats['Cm_rmse'],
     M_64_32_16_stats['CL_r2'], M_64_32_16_stats['CD_r2'], M_64_32_16_stats['Cm_r2']),
    
    (M_64_32_16_8_stats['CL_rmse'], M_64_32_16_8_stats['CD_rmse'], M_64_32_16_8_stats['Cm_rmse'],
     M_64_32_16_8_stats['CL_r2'], M_64_32_16_8_stats['CD_r2'], M_64_32_16_8_stats['Cm_r2'])
]

column_headers = pd.MultiIndex.from_tuples([
    ('RMSE', 'CL'), ('RMSE', 'CD'), ('RMSE', 'Cm'),
    ('R²', 'CL'), ('R²', 'CD'), ('R²', 'Cm')
])

row_index = pd.MultiIndex.from_tuples(
    zip(range(1, 5), architectures),
    names=['Case No.', 'Network Architecture']
)

table_1_df = pd.DataFrame(table_data, index=row_index, columns=column_headers)

display(table_1_df.style.set_table_styles(
    [{'selector': 'th', 'props': [('text-align', 'left')]},
     {'selector': 'td', 'props': [('text-align', 'left')]}]
).format(precision=6))


## Network Performance Due to Increasing Neurons

### Case 1: Model [64, 32, 16, 3]

In [22]:
# RMSE and R² metric scores on test data for Model M_64_32_16
M_64_32_16_metrics = []

for i in range(20):
    print(f"\nRun {i+1}/20 for Model M_64_32_16...")

    M_64_32_16 = Sequential([
        Input(shape=(X_train.shape[1],)),
        Dense(64, activation='relu'),
        Dense(32, activation='relu'),
        Dense(16, activation='relu'),
        Dense(y_train.shape[1], activation='linear')
    ])

    M_64_32_16.compile(
        optimizer=Adam(learning_rate=5e-4, beta_1=0.9, beta_2=0.999),
        loss='mse',
        metrics=[RootMeanSquaredError()]
    )

    M_64_32_16.fit(
        X_train, y_train,
        epochs=50, batch_size=128,
        validation_data=(X_val, y_val),
        callbacks=[reduce_lr],
        verbose=0
    )

    y_pred = M_64_32_16.predict(X_test, verbose=0)
    r2 = r2_score(y_test, y_pred, multioutput='raw_values')
    rmse = np.sqrt(mean_squared_error(y_test, y_pred, multioutput='raw_values'))

    M_64_32_16_metrics.append({
        'CL_rmse': rmse[0], 'CD_rmse': rmse[1], 'Cm_rmse': rmse[2],
        'CL_r2': r2[0], 'CD_r2': r2[1], 'Cm_r2': r2[2]
    })

results_M_64_32_16_df = pd.DataFrame(M_64_32_16_metrics)

print(results_M_64_32_16_df.mean())



Run 1/20 for Model M_64_32_16...

Run 2/20 for Model M_64_32_16...

Run 3/20 for Model M_64_32_16...

Run 4/20 for Model M_64_32_16...

Run 5/20 for Model M_64_32_16...

Run 6/20 for Model M_64_32_16...

Run 7/20 for Model M_64_32_16...

Run 8/20 for Model M_64_32_16...

Run 9/20 for Model M_64_32_16...

Run 10/20 for Model M_64_32_16...

Run 11/20 for Model M_64_32_16...

Run 12/20 for Model M_64_32_16...

Run 13/20 for Model M_64_32_16...

Run 14/20 for Model M_64_32_16...

Run 15/20 for Model M_64_32_16...

Run 16/20 for Model M_64_32_16...

Run 17/20 for Model M_64_32_16...

Run 18/20 for Model M_64_32_16...

Run 19/20 for Model M_64_32_16...

Run 20/20 for Model M_64_32_16...
CL_rmse    0.003221
CD_rmse    0.004070
Cm_rmse    0.001648
CL_r2      0.999988
CD_r2      0.865080
Cm_r2      0.999786
dtype: float32


### Case 2: Model [128, 64, 32, 3]

In [23]:
# RMSE and R² metric scores on test data for Model M_128_64_32
M_128_64_32_metrics = []

for i in range(20):
    print(f"\nRun {i+1}/20 for Model M_128_64_32...")

    M_128_64_32 = Sequential([
        Input(shape=(X_train.shape[1],)),
        Dense(128, activation='relu'),
        Dense(64, activation='relu'),
        Dense(32, activation='relu'),
        Dense(y_train.shape[1], activation='linear')
    ])

    M_128_64_32.compile(
        optimizer=Adam(learning_rate=5e-4, beta_1=0.9, beta_2=0.999),
        loss='mse',
        metrics=[RootMeanSquaredError()]
    )

    M_128_64_32.fit(
        X_train, y_train,
        epochs=50, batch_size=128,
        validation_data=(X_val, y_val),
        callbacks=[reduce_lr],
        verbose=0
    )

    y_pred = M_128_64_32.predict(X_test, verbose=0)
    r2 = r2_score(y_test, y_pred, multioutput='raw_values')
    rmse = np.sqrt(mean_squared_error(y_test, y_pred, multioutput='raw_values'))

    M_128_64_32_metrics.append({
        'CL_rmse': rmse[0], 'CD_rmse': rmse[1], 'Cm_rmse': rmse[2],
        'CL_r2': r2[0], 'CD_r2': r2[1], 'Cm_r2': r2[2]
    })

results_M_128_64_32_df = pd.DataFrame(M_128_64_32_metrics)

print(results_M_128_64_32_df.mean())



Run 1/20 for Model M_128_64_32...

Run 2/20 for Model M_128_64_32...

Run 3/20 for Model M_128_64_32...

Run 4/20 for Model M_128_64_32...

Run 5/20 for Model M_128_64_32...

Run 6/20 for Model M_128_64_32...

Run 7/20 for Model M_128_64_32...

Run 8/20 for Model M_128_64_32...

Run 9/20 for Model M_128_64_32...

Run 10/20 for Model M_128_64_32...

Run 11/20 for Model M_128_64_32...

Run 12/20 for Model M_128_64_32...

Run 13/20 for Model M_128_64_32...

Run 14/20 for Model M_128_64_32...

Run 15/20 for Model M_128_64_32...

Run 16/20 for Model M_128_64_32...

Run 17/20 for Model M_128_64_32...

Run 18/20 for Model M_128_64_32...

Run 19/20 for Model M_128_64_32...

Run 20/20 for Model M_128_64_32...
CL_rmse    0.002847
CD_rmse    0.003759
Cm_rmse    0.001304
CL_r2      0.999990
CD_r2      0.884898
Cm_r2      0.999866
dtype: float32


### Case 3: Model [256, 128, 64, 3]

In [24]:
# RMSE and R² metric scores on test data for Model M_256_128_64
M_256_128_64_metrics = []

for i in range(20):
    print(f"\nRun {i+1}/20 for Model M_256_128_64...")

    M_256_128_64 = Sequential([
        Input(shape=(X_train.shape[1],)),
        Dense(256, activation='relu'),
        Dense(128, activation='relu'),
        Dense(64, activation='relu'),
        Dense(y_train.shape[1], activation='linear')
    ])

    M_256_128_64.compile(
        optimizer=Adam(learning_rate=5e-4, beta_1=0.9, beta_2=0.999),
        loss='mse',
        metrics=[RootMeanSquaredError()]
    )

    M_256_128_64.fit(
        X_train, y_train,
        epochs=50, batch_size=128,
        validation_data=(X_val, y_val),
        callbacks=[reduce_lr],
        verbose=0
    )

    y_pred = M_256_128_64.predict(X_test, verbose=0)
    r2 = r2_score(y_test, y_pred, multioutput='raw_values')
    rmse = np.sqrt(mean_squared_error(y_test, y_pred, multioutput='raw_values'))

    M_256_128_64_metrics.append({
        'CL_rmse': rmse[0], 'CD_rmse': rmse[1], 'Cm_rmse': rmse[2],
        'CL_r2': r2[0], 'CD_r2': r2[1], 'Cm_r2': r2[2]
    })

results_M_256_128_64_df = pd.DataFrame(M_256_128_64_metrics)

print(results_M_256_128_64_df.mean())



Run 1/20 for Model M_256_128_64...

Run 2/20 for Model M_256_128_64...

Run 3/20 for Model M_256_128_64...

Run 4/20 for Model M_256_128_64...

Run 5/20 for Model M_256_128_64...

Run 6/20 for Model M_256_128_64...

Run 7/20 for Model M_256_128_64...

Run 8/20 for Model M_256_128_64...

Run 9/20 for Model M_256_128_64...

Run 10/20 for Model M_256_128_64...

Run 11/20 for Model M_256_128_64...

Run 12/20 for Model M_256_128_64...

Run 13/20 for Model M_256_128_64...

Run 14/20 for Model M_256_128_64...

Run 15/20 for Model M_256_128_64...

Run 16/20 for Model M_256_128_64...

Run 17/20 for Model M_256_128_64...

Run 18/20 for Model M_256_128_64...

Run 19/20 for Model M_256_128_64...

Run 20/20 for Model M_256_128_64...
CL_rmse    0.002647
CD_rmse    0.003550
Cm_rmse    0.001043
CL_r2      0.999991
CD_r2      0.897266
Cm_r2      0.999913
dtype: float32


### Case 4: Model [512, 256, 128, 3]

In [25]:
# RMSE and R² metric scores on test data for Model M_512_256_128
M_512_256_128_metrics = []

for i in range(20):
    print(f"\nRun {i+1}/20 for Model M_512_256_128...")

    M_512_256_128 = Sequential([
        Input(shape=(X_train.shape[1],)),
        Dense(512, activation='relu'),
        Dense(256, activation='relu'),
        Dense(128, activation='relu'),
        Dense(y_train.shape[1], activation='linear')
    ])

    M_512_256_128.compile(
        optimizer=Adam(learning_rate=5e-4, beta_1=0.9, beta_2=0.999),
        loss='mse',
        metrics=[RootMeanSquaredError()]
    )

    M_512_256_128.fit(
        X_train, y_train,
        epochs=50, batch_size=128,
        validation_data=(X_val, y_val),
        callbacks=[reduce_lr],
        verbose=0
    )

    y_pred = M_512_256_128.predict(X_test, verbose=0)
    r2 = r2_score(y_test, y_pred, multioutput='raw_values')
    rmse = np.sqrt(mean_squared_error(y_test, y_pred, multioutput='raw_values'))

    M_512_256_128_metrics.append({
        'CL_rmse': rmse[0], 'CD_rmse': rmse[1], 'Cm_rmse': rmse[2],
        'CL_r2': r2[0], 'CD_r2': r2[1], 'Cm_r2': r2[2]
    })

results_M_512_256_128_df = pd.DataFrame(M_512_256_128_metrics)

print(results_M_512_256_128_df.mean())



Run 1/20 for Model M_512_256_128...

Run 2/20 for Model M_512_256_128...

Run 3/20 for Model M_512_256_128...

Run 4/20 for Model M_512_256_128...

Run 5/20 for Model M_512_256_128...

Run 6/20 for Model M_512_256_128...

Run 7/20 for Model M_512_256_128...

Run 8/20 for Model M_512_256_128...

Run 9/20 for Model M_512_256_128...

Run 10/20 for Model M_512_256_128...

Run 11/20 for Model M_512_256_128...

Run 12/20 for Model M_512_256_128...

Run 13/20 for Model M_512_256_128...

Run 14/20 for Model M_512_256_128...

Run 15/20 for Model M_512_256_128...

Run 16/20 for Model M_512_256_128...

Run 17/20 for Model M_512_256_128...

Run 18/20 for Model M_512_256_128...

Run 19/20 for Model M_512_256_128...

Run 20/20 for Model M_512_256_128...
CL_rmse    0.002013
CD_rmse    0.003299
Cm_rmse    0.000912
CL_r2      0.999995
CD_r2      0.911276
Cm_r2      0.999931
dtype: float32


### Case 5: Model [1024, 512, 256, 3]

In [26]:
# RMSE and R² metric scores on test data for Model M_1024_512_256
M_1024_512_256_metrics = []

for i in range(20):
    print(f"\nRun {i+1}/20 for Model M_1024_512_256...")

    M_1024_512_256 = Sequential([
        Input(shape=(X_train.shape[1],)),
        Dense(1024, activation='relu'),
        Dense(512, activation='relu'),
        Dense(256, activation='relu'),
        Dense(y_train.shape[1], activation='linear')
    ])

    M_1024_512_256.compile(
        optimizer=Adam(learning_rate=5e-4, beta_1=0.9, beta_2=0.999),
        loss='mse',
        metrics=[RootMeanSquaredError()]
    )

    M_1024_512_256.fit(
        X_train, y_train,
        epochs=50, batch_size=128,
        validation_data=(X_val, y_val),
        callbacks=[reduce_lr],
        verbose=0
    )

    y_pred = M_1024_512_256.predict(X_test, verbose=0)
    r2 = r2_score(y_test, y_pred, multioutput='raw_values')
    rmse = np.sqrt(mean_squared_error(y_test, y_pred, multioutput='raw_values'))

    M_1024_512_256_metrics.append({
        'CL_rmse': rmse[0], 'CD_rmse': rmse[1], 'Cm_rmse': rmse[2],
        'CL_r2': r2[0], 'CD_r2': r2[1], 'Cm_r2': r2[2]
    })

results_M_1024_512_256_df = pd.DataFrame(M_1024_512_256_metrics)

print(results_M_1024_512_256_df.mean())



Run 1/20 for Model M_1024_512_256...

Run 2/20 for Model M_1024_512_256...

Run 3/20 for Model M_1024_512_256...

Run 4/20 for Model M_1024_512_256...

Run 5/20 for Model M_1024_512_256...

Run 6/20 for Model M_1024_512_256...

Run 7/20 for Model M_1024_512_256...

Run 8/20 for Model M_1024_512_256...

Run 9/20 for Model M_1024_512_256...

Run 10/20 for Model M_1024_512_256...

Run 11/20 for Model M_1024_512_256...

Run 12/20 for Model M_1024_512_256...

Run 13/20 for Model M_1024_512_256...

Run 14/20 for Model M_1024_512_256...

Run 15/20 for Model M_1024_512_256...

Run 16/20 for Model M_1024_512_256...

Run 17/20 for Model M_1024_512_256...

Run 18/20 for Model M_1024_512_256...

Run 19/20 for Model M_1024_512_256...

Run 20/20 for Model M_1024_512_256...
CL_rmse    0.002175
CD_rmse    0.003238
Cm_rmse    0.000879
CL_r2      0.999994
CD_r2      0.914467
Cm_r2      0.999935
dtype: float32


In [27]:
M_64_32_16_stats = results_M_64_32_16_df.mean()
M_128_64_32_stats = results_M_128_64_32_df.mean()
M_256_128_64_stats = results_M_256_128_64_df.mean()
M_512_256_128_stats = results_M_512_256_128_df.mean()
M_1024_512_256_stats = results_M_1024_512_256_df.mean()

architectures = [
    '64, 32, 16, 3',
    '128, 64, 32, 3',
    '256, 128, 64, 3',
    '512, 256, 128, 3',
    '1024, 512, 256, 3'
]

table_data = [
    (M_64_32_16_stats['CL_rmse'], M_64_32_16_stats['CD_rmse'], M_64_32_16_stats['Cm_rmse'],
     M_64_32_16_stats['CL_r2'], M_64_32_16_stats['CD_r2'], M_64_32_16_stats['Cm_r2']),
    
    (M_128_64_32_stats['CL_rmse'], M_128_64_32_stats['CD_rmse'], M_128_64_32_stats['Cm_rmse'],
     M_128_64_32_stats['CL_r2'], M_128_64_32_stats['CD_r2'], M_128_64_32_stats['Cm_r2']),
    
    (M_256_128_64_stats['CL_rmse'], M_256_128_64_stats['CD_rmse'], M_256_128_64_stats['Cm_rmse'],
     M_256_128_64_stats['CL_r2'], M_256_128_64_stats['CD_r2'], M_256_128_64_stats['Cm_r2']),
    
    (M_512_256_128_stats['CL_rmse'], M_512_256_128_stats['CD_rmse'], M_512_256_128_stats['Cm_rmse'],
     M_512_256_128_stats['CL_r2'], M_512_256_128_stats['CD_r2'], M_512_256_128_stats['Cm_r2']),
    
    (M_1024_512_256_stats['CL_rmse'], M_1024_512_256_stats['CD_rmse'], M_1024_512_256_stats['Cm_rmse'],
     M_1024_512_256_stats['CL_r2'], M_1024_512_256_stats['CD_r2'], M_1024_512_256_stats['Cm_r2'])
]

column_headers = pd.MultiIndex.from_tuples([
    ('RMSE', 'CL'), ('RMSE', 'CD'), ('RMSE', 'Cm'),
    ('R²', 'CL'), ('R²', 'CD'), ('R²', 'Cm')
])

row_index = pd.MultiIndex.from_tuples(
    zip(range(1, 6), architectures),
    names=['Case No.', 'Network Architecture']
)

table_1_df = pd.DataFrame(table_data, index=row_index, columns=column_headers)

display(table_1_df.style.set_table_styles(
    [{'selector': 'th', 'props': [('text-align', 'left')]},
     {'selector': 'td', 'props': [('text-align', 'left')]}]
).format(precision=6))
